In [ ]:
#User will be prompted to upload the master file. This will be the exact file as extracted from https://data.ontario.ca/dataset/university-enrolment
#Note: the downloaded workbook should be renamed to University Enrolment.xlsx
from google.colab import files
data = files.upload()

Saving University Enrolment.xlsx to University Enrolment.xlsx


#Libraries

In [1]:
# Import all the relevant libraries
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

#Importing Data Sets

In [2]:
# Note: openpyxl must be installed through -pip install openpyxl in order to read .xlsx files
university = pd.ExcelFile('University Enrolment.xlsx')
country=pd.read_excel(university, 'COUNTRY', skiprows=[0])
program=pd.read_excel(university, 'CIP PROGRAM',skiprows=[0])

In [3]:
# Fill empty header columns with placeholder names for later data cleaning
country.iloc[0,1]='1'
country.iloc[0,2]='2'

In [4]:
# Change the headers of the columns to the appropriate names and delete irrelevant columns
header = country.loc[0:1].fillna('').apply(' '.join)
country.columns = header
country = country.drop([0,1]).reset_index()
country = country.drop(country.columns[0],axis=1)
country.rename(columns={country.columns[0]:'University'}, inplace=True)
country.rename(columns={country.columns[1]:'Country'}, inplace=True)
country = country.drop(country.columns[2],axis=1)

In [5]:
# Drop the last row of the worksheet, which is the date information
program.drop(program.tail(1).index,inplace=True)

In [6]:
# Fill empty header columns with placeholder names for later data cleaning
program.iloc[0,1]='1'
program.iloc[0,2]='2'
program.iloc[0,3]='3'

In [7]:
# Change the headers of the columns to the appropriate names and delete irrelevant columns
header = program.loc[0:1].fillna('').apply(' '.join)
program.columns = header
program = program.drop([0,1]).reset_index()
program = program.drop(program.columns[0],axis=1)
program.rename(columns={program.columns[0]:'University'}, inplace=True)
program.rename(columns={program.columns[2]:'Program'}, inplace=True)
program = program.drop(program.columns[1],axis=1)
program = program.drop(program.columns[2],axis=1)

#Data Cleaning and Manipulation

##Country

In [8]:
# Replace missing numbers with 5, which is the median value for overall headcounts, and replace - with 0
country = country[country['Country'].notna()]
country.fillna(5,inplace=True) 
country.replace(('-'),(0), inplace=True)
country[["2012-2013 Undergraduate", "2012-2013 Graduate","2013-2014 Undergraduate","2013-2014 Graduate","2014-2015 Undergraduate","2014-2015 Graduate","2015-2016 Undergraduate","2015-2016 Graduate","2016-2017 Undergraduate","2016-2017 Graduate","2017-2018 Undergraduate","2017-2018 Graduate","2018-2019 Undergraduate","2018-2019 Graduate","2019-2020 Undergraduate","2019-2020 Graduate"]] = country[["2012-2013 Undergraduate", "2012-2013 Graduate","2013-2014 Undergraduate","2013-2014 Graduate","2014-2015 Undergraduate","2014-2015 Graduate","2015-2016 Undergraduate","2015-2016 Graduate","2016-2017 Undergraduate","2016-2017 Graduate","2017-2018 Undergraduate","2017-2018 Graduate","2018-2019 Undergraduate","2018-2019 Graduate","2019-2020 Undergraduate","2019-2020 Graduate"]].apply(pd.to_numeric)
country

,University,Country,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,AFGHANISTAN,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
1,Algoma,ANGOLA,0,0,5,0,0,0,0,0,5,0,5,0,5,0,5,0
2,Algoma,ANTIGUA AND BARBUDA,0,0,0,0,5,0,5,0,5,0,0,0,5,0,5,0
3,Algoma,AUSTRALIA,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
4,Algoma,AUSTRIA,0,0,5,0,5,0,5,0,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,York,VIET NAM,129,5,113,5,112,5,131,5,116,13,132,12,181,10,221,16
3183,York,VIRGIN ISLANDS U.S.,0,0,0,0,0,0,5,0,5,0,5,0,5,0,0,0
3184,York,YEMEN,5,0,5,0,11,5,5,0,5,0,5,0,17,5,26,5
3185,York,ZAMBIA,5,5,5,0,5,0,5,0,5,0,5,0,5,0,5,0


In [9]:
# Rename the countries to be compatible with google trend country names and to future error handling
country['Country'] = country['Country'].str.title()
country['Country'].replace({
"Iran" : "Iran, Islamic Republic of",
"United States Minor Outlying Islands" : "United States",
"America (Country Not Specified)" : "Removed",
"Outdated: Czechoslovakia Republic" : "Czechia",
"St Kitts and Nevis" : "Saint Kitts and Nevis",
"Yemen, Democratic" : "Yemen",
"Wallis And Futuna" : "France",
"Cocos (Keeling) Islands" : "Australia",
"Outdated: Germany, Dem.rep (East)" : "Germany",
"Korea South" : "Korea, Republic of",
"Congo, Democratic Republic Of The" : "Congo",
"Zaire" : "Congo",
"Ireland Republic Of (Eire)" : "Ireland",
"Antigua And Barbuda" : "Antigua and Barbuda",
"Trinidad And Tobago" : "Trinidad and Tobago",
"Bosnia And Herzegovina" : "Bosnia and Herzegovina",
"Outdated: Vietnam, Republic Of" : "Viet Nam",
"Palestinian Territory Occupied" : "Palestine, State of",
"Russia" : "Russian Federation",
"St Lucia" : "Saint Lucia",
"Syria" : "Syrian Arab Republic",
"Taiwan" : "Taiwan, Province of China",
"Tanzania United Republic Of" : "Tanzania, United Republic of",
"Venezuela" : "Venezuela, Bolivarian Republic of",
"Czech Republic" : "Czechia",
"England" : "United Kingdom",
"Macau" : "Macao", 
"Wallis And Futuna" : "Wallis and Futuna",
"Turks And Caicos Islands" : "Turks and Caicos Islands",
"Moldova, Republic Of" : "Moldova, Republic of",
"Netherlands Antilles" : "Netherlands",
"Not Reported" : "Removed",
"Palestinian Territory Ocuppied" : "Palestine, State of",
"Scotland" : "United Kingdom",
"Serbia And Montenegro" : "Serbia",
"Bolivia" : "Bolivia, Plurinational State of",
"Northern Ireland" : "United Kingdom",
"Swaziland" : "Eswatini",
"Stateless" : "Removed",
"Not Applicable" : "Removed",
"Heard Island And Mcdonald Islands" : "Heard Island And Mcdonald Islands",
"Sao Tome And Principe" : "Sao Tome and Principe",
"St Martin" : "Saint Martin (French part)",
"Cape Verde" : "Cabo Verde",
"Côte D'Ivoire" : "Côte d'Ivoire",
"Macedonia (Fyrom)" : "North Macedonia",
"St Vincent And The Grenadines":"Saint Vincent and the Grenadines",
"Africa (Country N.R.)":"Removed",
"Asia (Country N.R.)":"Removed",
"Kosovo":"Removed",
"Russia (Old U.S.S.R.)":"Russian Federation",
"St Kitts And Nevis":"Saint Kitts and Nevis",
"Channel Islands":"United Kingdom",
"Korea North":"Korea, Democratic People's Republic of",
"Outdated: Germany, Dem.Rep (East)":"Germany",
"St Helena":"Saint Helena, Ascension and Tristan da Cunha",
"British West Indies":"United Kingdom",
"St Barthélemy":"Saint Barthélemy",
"Virgin Islands British":"Virgin Islands, British",
"Heard Island And Mcdonald Islands":"Heard Island and McDonald Islands",
"Laos":"Lao People's Democratic Republic",
"Sint Maarten (Dutch Part)":"Sint Maarten (Dutch part)",
"Virgin Islands U.S.":"Virgin Islands, U.S.",
"Wales" : "United Kingdom"}, inplace=True)
country.drop(country.loc[country['Country']=='Removed'].index, inplace=True)
country

,University,Country,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,Afghanistan,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
1,Algoma,Angola,0,0,5,0,0,0,0,0,5,0,5,0,5,0,5,0
2,Algoma,Antigua and Barbuda,0,0,0,0,5,0,5,0,5,0,0,0,5,0,5,0
3,Algoma,Australia,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
4,Algoma,Austria,0,0,5,0,5,0,5,0,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3182,York,Viet Nam,129,5,113,5,112,5,131,5,116,13,132,12,181,10,221,16
3183,York,"Virgin Islands, U.S.",0,0,0,0,0,0,5,0,5,0,5,0,5,0,0,0
3184,York,Yemen,5,0,5,0,11,5,5,0,5,0,5,0,17,5,26,5
3185,York,Zambia,5,5,5,0,5,0,5,0,5,0,5,0,5,0,5,0


In [10]:
# Group the data by University and Country just in case there were 2 rows with the same names
country = country.groupby(['University','Country']).sum().reset_index()
country

,University,Country,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,Afghanistan,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
1,Algoma,Angola,0,0,5,0,0,0,0,0,5,0,5,0,5,0,5,0
2,Algoma,Antigua and Barbuda,0,0,0,0,5,0,5,0,5,0,0,0,5,0,5,0
3,Algoma,Australia,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0
4,Algoma,Austria,0,0,5,0,5,0,5,0,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,York,Viet Nam,129,5,113,5,112,5,131,5,116,13,132,12,181,10,221,16
3021,York,"Virgin Islands, U.S.",0,0,0,0,0,0,5,0,5,0,5,0,5,0,0,0
3022,York,Yemen,5,0,5,0,11,5,5,0,5,0,5,0,17,5,26,5
3023,York,Zambia,5,5,5,0,5,0,5,0,5,0,5,0,5,0,5,0


In [11]:
#Data validation. Ensure no duplicate rows. The first row should always have a 1 in "count". This will ensure that there is only one combination of University and Country
test=country.groupby(['University','Country'],as_index=False).agg(
    count=('Country','count'))
test = test.sort_values(['count'], ascending = (False))
test

,University,Country,count
0,Algoma,Afghanistan,1
2021,Trent,Netherlands,1
2012,Trent,Maldives,1
2013,Trent,Malta,1
2014,Trent,Mauritius,1
...,...,...,...
1010,McMaster,"Venezuela, Bolivarian Republic of",1
1011,McMaster,Viet Nam,1
1012,McMaster,Wallis and Futuna,1
1013,McMaster,Yemen,1


In [12]:
# Ensure there are no missing values in the dataset
country.isnull().sum() 

University                 0
Country                    0
2012-2013 Undergraduate    0
2012-2013 Graduate         0
2013-2014 Undergraduate    0
2013-2014 Graduate         0
2014-2015 Undergraduate    0
2014-2015 Graduate         0
2015-2016 Undergraduate    0
2015-2016 Graduate         0
2016-2017 Undergraduate    0
2016-2017 Graduate         0
2017-2018 Undergraduate    0
2017-2018 Graduate         0
2018-2019 Undergraduate    0
2018-2019 Graduate         0
2019-2020 Undergraduate    0
2019-2020 Graduate         0
dtype: int64

In [13]:
# Ensure the data type is correct
country.dtypes

University                 object
Country                    object
2012-2013 Undergraduate     int64
2012-2013 Graduate          int64
2013-2014 Undergraduate     int64
2013-2014 Graduate          int64
2014-2015 Undergraduate     int64
2014-2015 Graduate          int64
2015-2016 Undergraduate     int64
2015-2016 Graduate          int64
2016-2017 Undergraduate     int64
2016-2017 Graduate          int64
2017-2018 Undergraduate     int64
2017-2018 Graduate          int64
2018-2019 Undergraduate     int64
2018-2019 Graduate          int64
2019-2020 Undergraduate     int64
2019-2020 Graduate          int64
dtype: object

##Program

In [14]:
# Group programs into their respective fields of study.
program['Program'] = program['Program'].str.capitalize()
program['Program'].replace({
"Environmental science":"Environmental Studies",
"Computer science":"Computer science",
"Chemical engineering":"Engineering",
"Mechanical engineering":"Engineering",
"Linguistic, comparative and related language studies and services, other":"Linguistics",
"Law (llb, jd, bcl)":"Law",
"English language and literature, general":"Linguistics",
"General studies":"General studies",
"Biology/biological sciences, general":"Biologies",
"Mathematics, general":"Statistics",
"Multidisciplinary/interdisciplinary studies, other":"General studies",
"Psychology, general":"Philosophy",
"Social work, general":"Social Studies",
"Economics, general":"Economics",
"Geography":"Geographies",
"Political science and government, general":"Social Studies",
"Sociology":"Social Studies",
"Visual and performing arts, general":"Arts",
"Fine/studio arts, general":"Arts",
"Fine arts and art studies, other":"Arts",
"Music, general":"Arts",
"Business/commerce, general":"Business",
"Business administration and management, general":"Business",
"Business/managerial economics":"Business",
"History, general":"History",
"French language and literature, generalcan":"Linguistics",
"Agricultural production operations, other":"Environmental Studies",
"Latin american studies":"Cultural Studies",
"Canadian studies":"Cultural Studies",
"Aboriginal studies":"Cultural Studies",
"Women's studies":"Cultural Studies",
"Speech communication and rhetoric":"Communication Studies",
"Mass communication/media studies":"Communication Studies",
"Organizational communication, general":"Communication Studies",
"Computer systems networking and telecommunications":"Computer science",
"Education, general":"Teacher Education",
"Aboriginal education":"Teacher Education",
"Curriculum and instruction":"Teacher Education",
"Educational leadership and administration, general":"Teacher Education",
"Social and philosophical foundations of education":"Teacher Education",
"Adult and continuing education and teaching":"Teacher Education",
"Elementary education and teaching":"Teacher Education",
"Junior high/intermediate/middle school education and teaching":"Teacher Education",
"Secondary education and teaching":"Teacher Education",
"Technology teacher education/industrial arts teacher education":"Teacher Education",
"Teaching english as a second or foreign language/esl language instructor":"Teacher Education",
"Linguistics":"Linguistics",
"Comparative literature":"Linguistics",
"Italian language and literature":"Linguistics",
"Spanish language and literature":"Linguistics",
"Classics and classical languages, literatures and linguistics, general":"Linguistics",
"English as a second languagecan":"Linguistics",
"Child development":"Psychologies",
"Family and community services":"Social Studies",
"Child care provider/assistant":"Healthcare Studies",
"English speech and rhetorical studies":"Linguistics",
"English technical and business writing":"Linguistics",
"English creative writing":"Linguistics",
"English rhetoric and composition/writing studies, other":"Linguistics",
"Liberal arts and sciences/liberal studies":"General studies",
"Humanities/humanistic studies":"History",
"Liberal arts and sciences, general studies and humanities, other":"General studies",
"Biomedical sciences, general":"Biologies",
"Biochemistry":"Biologies",
"Biophysics":"Biologies",
"Botany/plant biology, general":"Biologies",
"Cell/cellular and molecular biology":"Biologies",
"Neurobiology and neurophysiology":"Biologies",
"Biotechnology":"Biologies",
"Ecology":"Biologies",
"Mathematical statistics and probability":"Statistics",
"Mathematics and computer science":"Statistics",
"Medieval and renaissance studies":"History",
"Behavioural sciences":"Psychologies",
"Classical, ancient mediterranean and near eastern studies and archaeology":"History",
"Intercultural/multicultural and diversity studies":"General studies",
"Neuroscience":"Biologies",
"Sustainability studies":"Environmental Studies",
"Parks, recreation and leisure studies":"Health Science",
"Health and physical education/fitness, general":"Health Science",
"Sport and fitness administration/management":"Health Science",
"Kinesiology and exercise science":"Health Science",
"Philosophy":"Philosophy",
"Philosophy, logic and ethics, other":"Philosophy",
"Chemistry, general":"Sciences",
"Inorganic chemistry":"Sciences",
"Organic chemistry":"Sciences",
"Physical chemistry":"Sciences",
"Geology/earth science, general":"Sciences",
"Physics, general":"Sciences",
"Condensed matter and materials physics":"Sciences",
"Theoretical and mathematical physics":"Sciences",
"Physics, other":"Sciences",
"Materials science":"Sciences",
"Developmental and child psychology":"Psychologies",
"Social psychology":"Psychologies",
"Educational psychology":"Psychologies",
"Public administration":"Social Studies",
"Public policy analysis, general":"Social Studies",
"Social sciences, general":"Social Studies",
"Applied economics":"Economics",
"Development economics and international development":"Economics",
"International relations and affairs":"Social Studies",
"Canadian government and politics":"Social Studies",
"Social sciences, other":"Social Studies",
"Game and interactive media design":"Arts",
"Drama and dramatics/theatre arts, general":"Arts",
"Film/cinema/video studies":"Arts",
"Cinematography and film/video production":"Arts",
"Art history, criticism and conservation":"Arts",
"Intermedia/multimedia":"Arts",
"Visual and performing arts, other":"Arts",
"Health services/allied health/health sciences, general":"Healthcare Studies",
"Audiology/audiologist":"Healthcare Studies",
"Communication disorders sciences and services, other":"Healthcare Studies",
"Nursing/registered nurse (rn, asn, bscn, mscn)":"Healthcare Studies",
"Public health, general (mph, dph)":"Healthcare Studies",
"Public health education and promotion":"Healthcare Studies",
"Community health and preventive medicine":"Healthcare Studies",
"Public health, other":"Healthcare Studies",
"Therapeutic recreation/recreational therapy":"Healthcare Studies",
"Registered nursing/registered nurse (rn, asn, bsn, bscn, msn, mscn)":"Healthcare Studies",
"Registered nursing, nursing administration, nursing research and clinical nursing, other":"Healthcare Studies",
"Accounting":"Business",
"Accounting and business/management":"Business",
"Accounting and related services, other":"Business",
"Finance, general":"Business",
"Tourism and travel services management":"Business",
"Human resources management/personnel administration, general":"Business",
"Organizational behaviour studies":"Business",
"Labour studies":"Business",
"Management science":"Business",
"Marketing/marketing management, general":"Business",
"Taxation":"Business",
"Tourism promotion operations":"Business",
"Food science":"Environmental Studies",
"Environmental studies":"Environmental Studies",
"Natural resource economics":"Environmental Studies",
"Architecture (barch, ba, bs, bsc, march, ma, ms, msc, phd)":"Architectural Studies",
"City/urban, community and regional planning":"Architectural Studies",
"African studies":"Cultural Studies",
"Russian, central european, east european and eurasian studies":"Cultural Studies",
"European studies/civilization":"Cultural Studies",
"Regional studies (u.s., canadian, foreign)":"Cultural Studies",
"French studies":"Cultural Studies",
"Communication and media studies, other":"Communication Studies",
"Journalism, general":"Communication Studies",
"Broadcast journalism":"Communication Studies",
"Animation, interactive technology, video graphics and special effects":"Communication Studies",
"Information technology":"Computer science",
"Information science/studies":"Computer science",
"Web page, digital/multimedia and information resources design":"Computer science",
"Computer and information systems security/information assurance":"Computer science",
"Engineering, general":"Engineering",
"Aerospace, aeronautical and astronautical/space engineering":"Engineering",
"Bioengineering and biomedical engineering":"Engineering",
"Civil engineering, general":"Engineering",
"Computer engineering, general":"Engineering",
"Computer software engineering":"Engineering",
"Electrical and electronics engineering":"Engineering",
"Electrical, electronics and communications engineering, other":"Engineering",
"Engineering physics/applied physics":"Engineering",
"Environmental/environmental health engineering":"Engineering",
"Materials engineering":"Engineering",
"Systems engineering":"Engineering",
"Engineering, other":"Engineering",
"Computer software technology/technician":"Computer science",
"American sign language (asl)/langue des signes québécoise (lsq)":"Linguistics",
"Legal studies, general":"Law",
"Toxicology":"Biologies",
"Biostatistics":"Biologies",
"Bioinformatics":"Biologies",
"Applied mathematics, general":"Statistics",
"Computational mathematics":"Statistics",
"Statistics, general":"Statistics",
"Biological and physical sciences":"Sciences",
"Peace studies and conflict resolution":"Social Studies",
"Museology/museum studies":"History",
"Science, technology and society":"Social Studies",
"International/global studies":"Social Studies",
"Cognitive science":"Psychologies",
"Human computer interaction":"Computer science",
"Ethics":"Philosophy",
"Religion/religious studies, general":"Philosophy",
"Philosophy and religious studies, other":"Philosophy",
"Theology/theological studies":"Philosophy",
"Chemistry, other":"Sciences",
"Paleontology":"History",
"Industrial and organizational psychology":"Psychologies",
"Forensic science and technology":"Psychologies",
"Anthropology, general":"History",
"Criminology":"Psychologies",
"Econometrics and quantitative economics":"Economics",
"International economics":"Economics",
"Geographic information science and cartography":"Geographies",
"Geography and cartography, other":"Geographies",
"Political science and government, other":"Social Studies",
"Industrial and product design":"Arts",
"Music, other":"Arts",
"Health/medical physics":"Healthcare Studies",
"Non-profit/public/organizational management":"Business",
"Entrepreneurship/entrepreneurial studies":"Business",
"International finance":"Business",
"International business/trade/commerce":"Business",
"Management information systems, general":"Business",
"Information resources management":"Business",
"Management sciences and quantitative methods, other":"Business",
"History, other":"History",
"Agriculture, general":"Environmental Studies",
"Agribusiness/agricultural business operations":"Environmental Studies",
"Agricultural economics":"Environmental Studies",
"Horse husbandry/equine science and management":"Environmental Studies",
"Agroecology and sustainable agriculture":"Environmental Studies",
"Applied horticulture/horticulture operations, general":"Environmental Studies",
"Turf and turfgrass management":"Environmental Studies",
"Agricultural and extension education services":"Environmental Studies",
"Animal sciences, general":"Environmental Studies",
"Food technology and processing":"Environmental Studies",
"Plant sciences, general":"Environmental Studies",
"Agronomy and crop science":"Environmental Studies",
"Agricultural and horticultural plant breeding":"Environmental Studies",
"Agriculture, agriculture operations and related sciences, other":"Environmental Studies",
"Natural resources management and policy, general":"Environmental Studies",
"Land use planning and management/development":"Architectural Studies",
"Landscape architecture (bs, bsc, bsla, bla, msla, mla, phd)":"Architectural Studies",
"Computer and information sciences, general":"Computer science",
"Agricultural engineering":"Engineering",
"Water resources engineering":"Engineering",
"Computer hardware engineering":"Engineering",
"Biological/biosystems engineering":"Engineering",
"Environmental control technologies/technicians, other":"Environmental Studies",
"Hispanic and latin american languages, literatures and linguistics, general":"Linguistics",
"Human nutrition":"Environmental Studies",
"Housing and human environments, general":"Environmental Studies",
"Adult development and aging":"Psychologies",
"Child care and support services management":"Social Studies",
"Human development, family studies and related services, other":"Psychologies",
"Molecular biology":"Biologies",
"Cell/cellular biology and histology":"Biologies",
"Microbiology, general":"Biologies",
"Zoology/animal biology, general":"Biologies",
"Wildlife biology":"Biologies",
"Pharmacology":"Biologies",
"Environmental toxicology":"Biologies",
"Environmental biology":"Biologies",
"Population biology":"Biologies",
"Ecology, evolution, systematics and population biology, other":"Biologies",
"Biological and biomedical sciences, other":"Biologies",
"Nutrition sciences":"Health Science",
"Physical sciences, general":"Sciences",
"Chemical physics":"Sciences",
"Physical sciences, other":"Sciences",
"Cognitive psychology and psycholinguistics":"Psychologies",
"Criminal justice/safety studies":"Psychologies",
"Criminal justice/police science":"Psychologies",
"Anthropology, other":"History",
"Theatre literature, history and criticism":"Arts",
"Veterinary/animal health technology/technician and veterinary assistant":"Healthcare Studies",
"Clinical laboratory science/medical technology/technologist":"Healthcare Studies",
"Pharmacy, pharmaceutical sciences and administration, other":"Healthcare Studies",
"Veterinary medicine (dvm)":"Healthcare Studies",
"Veterinary sciences/veterinary clinical sciences, general (cert., ms, msc, phd)":"Healthcare Studies",
"Veterinary pathology and pathobiology (cert., ms, msc, phd)":"Healthcare Studies",
"Veterinary biomedical and clinical sciences (cert., ms, msc, phd), other":"Healthcare Studies",
"Hospitality administration/management, general":"Business",
"Hotel/motel administration/management":"Business",
"Marketing research":"Business",
"Real estate":"Business",
"Instructional program not reported":"General studies",
"Language interpretation and translation":"Linguistics",
"Forestry, general":"Environmental Studies",
"Forest sciences and biology":"Environmental Studies",
"Forest management/forest resources management":"Environmental Studies",
"Art teacher education":"Teacher Education",
"English/english language arts teacher education":"Teacher Education",
"Mathematics teacher education":"Teacher Education",
"Music teacher education":"Teacher Education",
"Science teacher education/general science teacher education":"Teacher Education",
"Social science teacher education":"Teacher Education",
"Social studies teacher education":"Teacher Education",
"Biology teacher education":"Teacher Education",
"Chemistry teacher education":"Teacher Education",
"French language/french language arts teacher education":"Teacher Education",
"History teacher education":"Teacher Education",
"Physics teacher education":"Teacher Education",
"Geography teacher education":"Teacher Education",
"Psychology teacher education":"Teacher Education",
"Teacher education and professional development, specific subject areas, other":"Teacher Education",
"Biochemistry/biophysics and molecular biology, other":"Biologies",
"Gerontology":"Biologies",
"Natural sciences":"Sciences",
"Hydrology and water resources science":"Sciences",
"Clinical psychology":"Psychologies",
"Experimental psychology":"Psychologies",
"Geropsychology":"Psychologies",
"Archaeology":"History",
"Economics, other":"Economics",
"Family practice nurse/nurse practitioner":"Healthcare Studies",
"Public health/community nurse/nursing":"Healthcare Studies",
"Nursing, other":"Healthcare Studies",
"Business administration, management and operations, other":"Business",
"Labour and industrial relations":"Business",
"Mining and mineral engineering":"Engineering",
"Aboriginal and foreign languages, literatures and linguistics, other":"Linguistics",
"Pharmacology and toxicology, other":"Biologies",
"Human services, general":"Social Studies",
"Design and applied arts, other":"Arts",
"Audiology/audiologist and speech-language pathology/pathologist":"Healthcare Studies",
"Medical radiologic technology/science-radiation therapist":"Healthcare Studies",
"Emergency room/trauma nurse/nursing":"Healthcare Studies",
"Nursing practice":"Healthcare Studies",
"Health professions and related clinical sciences, other":"Healthcare Studies",
"Laboratory medicine residency programs":"Medical School",
"Ethnic, cultural minority, gender, and group studies, other":"Cultural Studies",
"Area, ethnic, cultural, gender, and group studies, other":"Cultural Studies",
"Communication, general":"Communication Studies",
"Digital communication and media/multimedia":"Communication Studies",
"Computer/information technology administration and management, other":"Computer science",
"Early childhood education and teaching":"Teacher Education",
"Education, other":"Teacher Education",
"Chemical and biomolecular engineering":"Engineering",
"Structural engineering":"Engineering",
"Computer engineering, other":"Engineering",
"Engineering mechanics":"Engineering",
"Nuclear engineering":"Engineering",
"Manufacturing engineering":"Engineering",
"Mechatronics, robotics, and automation engineering":"Engineering",
"Civil engineering technology/technician":"Engineering",
"Automation engineer technology/technician":"Engineering",
"Energy management and systems technology/technician":"Engineering",
"Manufacturing engineering technology/technician":"Engineering",
"Automotive engineering technology/technician":"Engineering",
"Engineering design":"Engineering",
"Financial mathematics":"Statistics",
"Christian studies":"Philosophy",
"Bible/biblical studies":"Philosophy",
"Divinity/ministry (bdiv, mdiv)":"Philosophy",
"Theological and ministerial studies, other":"Philosophy",
"Pastoral studies/counselling":"Philosophy",
"Astrophysics":"Sciences",
"Clinical child psychology":"Psychologies",
"Applied behaviour analysis":"Psychologies",
"Psychology, other":"Psychologies",
"Health policy analysis":"Social Studies",
"Social work, other":"Social Studies",
"Art/art studies, general":"Arts",
"Speech-language pathology/pathologist":"Healthcare Studies",
"Health/health care administration/management":"Healthcare Studies",
"Health and medical administrative services, other":"Healthcare Studies",
"Diagnostic medical sonography/sonographer and ultrasound technician":"Healthcare Studies",
"Physician assistant":"Healthcare Studies",
"Medicine (md)":"Healthcare Studies",
"Medical scientist (ms, msc, phd)":"Healthcare Studies",
"Psychoanalysis and psychotherapy":"Healthcare Studies",
"Maternal/child health and neonatal nurse/nursing":"Healthcare Studies",
"Nursing science (msc, phd)":"Healthcare Studies",
"Critical care nursing":"Healthcare Studies",
"Clinical, hospital and managed care pharmacy (ms, msc, phd)":"Healthcare Studies",
"International public health/international health":"Healthcare Studies",
"Health services administration":"Healthcare Studies",
"Occupational therapy/therapist":"Healthcare Studies",
"Physical therapy/therapist":"Healthcare Studies",
"Rehabilitation science":"Healthcare Studies",
"Rehabilitation and therapeutic professions, other":"Healthcare Studies",
"Direct entry midwifery (lm, cpm)":"Healthcare Studies",
"Primary health care nurse/nursing and family practice nurse/nursing":"Healthcare Studies",
"Nursing science (ms, msc, phd)":"Healthcare Studies",
"Critical care nurse/nursing":"Healthcare Studies",
"Logistics, materials, and supply chain management":"Business",
"Operations management and supervision":"Business",
"E-commerce/electronic commerce":"Business",
"Accounting and finance":"Business",
"Finance and financial management services, other":"Business",
"Actuarial science":"Statistics",
"Business, management, marketing and related support services, other":"Business",
"Allergies and immunology residency programs":"Medical School",
"Anesthesiology residency programs":"Medical School",
"Cardiology residency programs":"Medical School",
"Chemical pathology residency programs":"Medical School",
"Child/pediatric neurology residency programs":"Medical School",
"Child psychiatry residency programs":"Medical School",
"Critical care medicine residency programs":"Medical School",
"Dermatology residency programs":"Medical School",
"Diagnostic radiology residency programs":"Medical School",
"Emergency medicine residency programs":"Medical School",
"Endocrinology and metabolism residency programs":"Medical School",
"Family medicine residency programs":"Medical School",
"Forensic pathology residency programs":"Medical School",
"Gastroenterology residency programs":"Medical School",
"General surgery residency programs":"Medical School",
"Geriatric medicine residency programs":"Medical School",
"Hematology residency programs":"Medical School",
"Hematological pathology residency programs":"Medical School",
"Infectious disease residency programs":"Medical School",
"Internal medicine residency programs":"Medical School",
"Neonatal-perinatal medicine residency programs":"Medical School",
"Nephrology residency programs":"Medical School",
"Neurological surgery/neurosurgery residency programs":"Medical School",
"Neurology residency programs":"Medical School",
"Obstetrics and gynecology residency programs":"Medical School",
"Oncology residency programs":"Medical School",
"Ophthalmology residency programs":"Medical School",
"Orthopedics/orthopedic surgery residency programs":"Medical School",
"Otolaryngology residency programs":"Medical School",
"Pathology residency programs":"Medical School",
"Pediatric hemato-oncology residency programs":"Medical School",
"Pediatric surgery residency programs":"Medical School",
"Pediatrics residency programs":"Medical School",
"Physical and rehabilitation medicine residency programs":"Medical School",
"Plastic surgery residency programs":"Medical School",
"Psychiatry residency programs":"Medical School",
"Public health medicine residency programs":"Medical School",
"Radiation oncology residency programs":"Medical School",
"Rheumatology residency programs":"Medical School",
"Thoracic surgery residency programs":"Medical School",
"Urology residency programs":"Medical School",
"Vascular surgery residency programs":"Medical School",
"Medical residency programs, other":"Medical School",
"Allergy and immunology residency programs":"Medical School",
"Child neurology residency programs":"Medical School",
"Neurological surgery residency programs":"Medical School",
"Orthopedic surgery residency programs":"Medical School",
"Physical medicine and rehabilitation residency programs":"Medical School",
"Public health and general preventive medicine residency programs":"Medical School",
"Medical residency programs-general certificates, other":"Medical School",
"Cardiovascular disease residency programs":"Medical School",
"Child and adolescent psychiatry residency programs":"Medical School",
"Congenital cardiac surgery residency programs":"Medical School",
"Developmental-behavioural pediatrics residency programs":"Medical School",
"Endocrinology, diabetes and metabolism residency programs":"Medical School",
"Forensic psychiatry residency programs":"Medical School",
"Geriatric psychiatry residency programs":"Medical School",
"Gynecologic oncology residency programs":"Medical School",
"Hospice and palliative medicine residency programs":"Medical School",
"Interventional cardiology residency programs":"Medical School",
"Maternal and fetal medicine residency programs":"Medical School",
"Medical biochemical genetics residency programs":"Medical School",
"Medical microbiology residency programs":"Medical School",
"Medical oncology residency programs":"Medical School",
"Neuroradiology residency programs":"Medical School",
"Pediatric critical care medicine residency programs":"Medical School",
"Pediatric emergency medicine residency programs":"Medical School",
"Pediatric gastroenterology residency programs":"Medical School",
"Pediatric hematology-oncology residency programs":"Medical School",
"Pulmonary disease residency programs":"Medical School",
"Reproductive endocrinology/infertility residency programs":"Medical School",
"Medical residency programs-subspecialty certificates, other":"Medical School",
"Dental, medical and veterinary residency programs, other":"Medical School",
"Nothing":"General studies",
"Could be in way too many series (more than 3)":"General studies",
"Educational evaluation and research":"Teacher Education",
"Teaching assistants/aides, general":"Teacher Education",
"Banking and financial support services":"Business",
"Instructional program not applicable":"General studies",
"Environmental design/architecture":"Architectural Studies",
"Publishing":"Linguistics",
"Systems science and theory":"Health Science",
"Commercial and advertising art":"Arts",
"Graphic design":"Arts",
"Illustration":"Arts",
"Photography":"Arts",
"Sculpture":"Arts",
"Printmaking":"Arts",
"Public relations/image management":"Communication Studies",
"Counsellor education/school counselling and guidance services":"Teacher Education",
"Teacher education, multiple levels":"Teacher Education",
"Health occupations teacher education":"Teacher Education",
"Teaching french as a second or foreign language":"Teacher Education",
"Engineering/industrial management":"Engineering",
"Aboriginal and foreign languages and literatures, general":"Linguistics",
"Russian language and literature":"Linguistics",
"German language and literature":"Linguistics",
"Arabic language and literature":"Linguistics",
"French as a second languagecan":"Linguistics",
"Foods, nutrition and wellness studies, general":"Health Science",
"Advanced legal research/studies, general (llm, mcl, mli, msl, lld, jsd/sjd)":"Law",
"Legal professions and studies, other":"Law",
"English literature, general":"Linguistics",
"Library and information science":"Communication Studies",
"Immunology":"Biologies",
"Microbiology and immunology":"Biologies",
"Physiology, general":"Biologies",
"Epidemiology":"Biologies",
"Molecular medicine":"Biologies",
"Applied mathematics, other":"Statistics",
"Missions/missionary studies and missiology":"Philosophy",
"Religious education":"Philosophy",
"Pre-theology/pre-ministerial studies":"Philosophy",
"Pastoral counselling and specialized ministries, other":"Philosophy",
"Theology and religious vocations, other":"Philosophy",
"Community organization and advocacy":"Social Studies",
"Acting":"Arts",
"Music pedagogy":"Arts",
"Health and wellness, general":"Healthcare Studies",
"Health information/medical records administration/administrator":"Healthcare Studies",
"Ophthalmic laboratory technology/technician":"Healthcare Studies",
"Organizational leadership":"Business",
"Management information systems and services, other":"Business",
"French technical and business writing":"Linguistics",
"French professional, technical, business, and scientific writingcan":"Linguistics",
"Colon and rectal surgery residency programs":"Medical School",
"Nuclear medicine residency programs":"Medical School",
"Clinical cardiac electrophysiology residency programs":"Medical School",
"Pain medicine residency programs":"Medical School",
"Pediatric cardiology residency programs":"Medical School",
"Pediatric endocrinology residency programs":"Medical School",
"Pediatric nephrology residency programs":"Medical School",
"Pediatric radiology residency programs":"Medical School",
"Surgery of the hand residency programs":"Medical School",
"Transplant hepatology residency programs":"Medical School",
"Area studies, other":"Cultural Studies",
"Technical teacher education":"Teacher Education",
"Engineering science":"Engineering",
"Geological/geophysical engineering":"Engineering",
"Aboriginal languages, literatures and linguistics":"Linguistics",
"English language and literature/letters, other":"Linguistics",
"Anatomy":"Biologies",
"Microbiological sciences and immunology, other":"Biologies",
"Pathology/experimental pathology":"Biologies",
"Mathematics and statistics, other":"Statistics",
"Writing (not for credit)":"Linguistics",
"Astronomy":"Sciences",
"Digital arts, general":"Arts",
"Musical theatre":"Arts",
"Drama/theatre arts and stagecraft, other":"Arts",
"Pre-medicine/pre-medical studies":"Healthcare Studies",
"Pharmaceutical marketing and management":"Healthcare Studies",
"Architectural and building sciences/technology (barch, ba, bs, bsc, march, ma, ms, msc, phd)":"Architectural Studies",
"Architecture and related services, other":"Architectural Studies",
"Disability studies":"Cultural Studies",
"Radio and television":"Communication Studies",
"Sports communication":"Communication Studies",
"Communication, journalism and related programs, other":"Communication Studies",
"Graphic communications, general":"Communication Studies",
"Family and consumer sciences/home economics teacher education":"Teacher Education",
"Industrial engineering":"Engineering",
"Foods, nutrition and related services, other":"Health Science",
"Apparel and textiles, general":"Arts",
"Apparel and textile marketing management":"Business",
"Biochemistry and molecular biology":"Biologies",
"Dance, general":"Arts",
"Fashion/apparel design":"Arts",
"Interior design":"Arts",
"Technical theatre/theatre design and technology":"Arts",
"Film/video and photographic arts, other":"Arts",
"Arts, entertainment, and media management, general":"Arts",
"Nurse midwife/nursing midwifery":"Healthcare Studies",
"Environmental health":"Healthcare Studies",
"Occupational health and industrial hygiene":"Healthcare Studies",
"Dietetics/dietitian (rd)":"Healthcare Studies",
"Business administration, management and operations":"Business",
"Retail management":"Business",
"Management information systems and services":"Business",
"Forestry, other":"Environmental Studies",
"Architectural history and criticism":"Architectural Studies",
"American/united states studies/civilization":"Cultural Studies",
"Asian studies/civilization":"Cultural Studies",
"East asian studies":"Cultural Studies",
"Pacific area/pacific rim studies":"Cultural Studies",
"Scandinavian studies":"Cultural Studies",
"South asian studies":"Cultural Studies",
"Caribbean studies":"Cultural Studies",
"German studies":"Cultural Studies",
"Italian studies":"Cultural Studies",
"Polish studies":"Cultural Studies",
"Health communication":"Communication Studies",
"Computer and information sciences and support services, other":"Computer science",
"Higher education/higher education administration":"Teacher Education",
"International and comparative education":"Teacher Education",
"Teacher education and professional development, specific levels and methods, other":"Teacher Education",
"Civil engineering, other":"Engineering",
"Slavic languages, literatures and linguistics, general":"Linguistics",
"Polish language and literature":"Linguistics",
"Ukrainian language and literature":"Linguistics",
"Portuguese language and literature":"Linguistics",
"Romance languages, literatures and linguistics, other":"Linguistics",
"Ancient/classical greek language and literature":"Linguistics",
"Latin language and literature":"Linguistics",
"English professional, technical, business, and scientific writing":"Linguistics",
"Molecular biophysics":"Biologies",
"Developmental biology and embryology":"Biologies",
"Cell/cellular biology and anatomical sciences, other":"Biologies",
"Animal physiology":"Biologies",
"Animal behaviour and ethology":"Biologies",
"Genetics, general":"Biologies",
"Molecular genetics":"Biologies",
"Human/medical genetics":"Biologies",
"Genome sciences/genomics":"Biologies",
"Pharmacology and toxicology, integrated":"Biologies",
"Evolutionary biology":"Biologies",
"Conservation biology":"Biologies",
"Mathematics, other":"Statistics",
"Logic":"Philosophy",
"Buddhist studies":"Philosophy",
"Islamic studies":"Philosophy",
"Jewish/judaic studies":"Philosophy",
"Religious/sacred music":"Philosophy",
"Planetary astronomy and science":"Sciences",
"Astronomy and astrophysics, other":"Sciences",
"Geological and earth sciences/geosciences, other":"Sciences",
"Counselling psychology":"Psychologies",
"School psychology":"Psychologies",
"Urban studies/affairs":"Social Studies",
"Arts management":"Arts",
"Music performance, general":"Arts",
"Music theory and composition":"Arts",
"Musicology and ethnomusicology":"Arts",
"Voice and opera":"Arts",
"Dentistry (dds, dmd)":"Healthcare Studies",
"Dental clinical sciences, general (ms, msc, phd)":"Healthcare Studies",
"Oral biology and oral and maxillofacial pathology (ms, msc, phd)":"Healthcare Studies",
"Dental public health and education (cert., ms, msc, mph, phd, dph)":"Healthcare Studies",
"Endodontics/endodontology (cert., ms, msc, phd)":"Healthcare Studies",
"Oral/maxillofacial surgery (cert., ms, msc, phd)":"Healthcare Studies",
"Orthodontics/orthodontology (cert., ms, msc, phd)":"Healthcare Studies",
"Pediatric dentistry/pedodontics (cert., ms, msc, phd)":"Healthcare Studies",
"Periodontics/periodontology (cert., ms, msc, phd)":"Healthcare Studies",
"Prosthodontics/prosthodontology (cert., ms, msc, phd)":"Healthcare Studies",
"Advanced/graduate dentistry and oral sciences (cert., ms, msc, phd), other":"Healthcare Studies",
"Emergency medical technology/technician (emt paramedic)":"Healthcare Studies",
"Genetic counselling/counsellor":"Healthcare Studies",
"Mental and social health services and allied professions, other":"Healthcare Studies",
"Pharmacy (pharmd, bs, bsc, bpharm)":"Healthcare Studies",
"Auditing":"Business",
"Investments and securities":"Business",
"College/university preparatory programs":"General studies",
"High school equivalence certificate programs":"General studies",
"History and philosophy of science and technology":"Philosophy",
"Oral and maxillofacial surgery residency programs":"Medical School",
"Dental residency programs, other":"Medical School",
"Blood banking residency programs":"Medical School",
"Neuropathology residency programs":"Medical School",
"Nuclear radiology residency programs":"Medical School",
"Occupational medicine residency programs":"Medical School",
"Natural resources conservation and research, other":"Environmental Studies",
"Spanish and iberian studies":"Linguistics",
"Communications technologies/technicians and support services":"Communication Studies",
"Personal and culinary services":"General studies",
"Social work":"Social Studies",
"Educational/instructional technology":"Teacher Education",
"Nuclear/nuclear power technology/technician":"Sciences",
"Forensic psychology":"Psychologies",
"Aboriginal and foreign language teacher education":"Teacher Education",
"Architectural engineering":"Engineering",
"Operations research":"Engineering",
"Nanotechnology":"Engineering",
"Germanic languages, literatures and linguistics, general":"Linguistics",
"Family systems":"Psychologies",
"Ancient studies/civilization":"History",
"Parks, recreation and leisure facilities management, general":"Health Science",
"Materials sciences, other":"Sciences",
"Health/medical preparatory programs, other":"Healthcare Studies",
"Optometry (od)":"Healthcare Studies",
"Medical informatics":"Healthcare Studies",
"Data processing and data processing technology/technician":"Computer science",
"Japanese language and literature":"Linguistics",
"Family and consumer sciences/human sciences, general":"Social Studies",
"Family and consumer sciences/human sciences business services, other":"Social Studies",
"Energy, environment andnatural resources law (llm, ms, msc, lld, jsd/sjd)":"Law",
"Library science, other":"General studies",
"Cell biology and anatomy":"Biologies",
"Physiology, pathology and related sciences, other":"Biologies",
"Statistics, other":"Statistics",
"Geophysics and seismology":"Geographies",
"Science technologies/technicians, other":"Sciences",
"Public administration and social service professions, other":"Social Studies",
"Demography and population studies":"Social Studies",
"Aviation/airway management and operations":"General studies",
"Conducting":"Arts",
"Keyboard instruments":"Arts",
"Music management and merchandising":"Arts",
"Music management":"Arts",
"Orthodontics residency programs":"Medical School",
"Mathematics and statistics":"Statistics",
"Computer games and programming skills (not for credit)":"Computer science",
"International public policy analysis":"Social Studies",
"National security policy studies":"Social Studies",
"Music therapy/therapist":"Arts",
"Journalism, other":"Communication Studies",
"Drama and dance teacher education":"Teacher Education",
"Teaching english or french as a second or foreign language, other":"Teacher Education",
"Mechanical engineering related technologies/technicians, other":"Engineering",
"Human development and family studies, general":"Psychologies",
"Pre-law studies":"Law",
"Fine and studio arts management":"Arts",
"Palliative care nurse/nursing":"Healthcare Studies",
"African american/black studies":"Cultural Studies",
"Geotechnical and geoenvironmental engineering":"Engineering",
"Computer technology/computer systems technology":"Computer science",
"Modern greek language and literature":"Linguistics",
"Legal research and advanced professional studies (post-llb/jd), other":"Law",
"Atmospheric sciences and meteorology, other":"Sciences",
"Security and protective services, other":"General studies",
"Design and visual communications, general":"Arts",
"Medical radiologic technology/science - radiation therapist":"Medical School",
"Medical residency programs - general certificates, other":"Medical School",
"Medical residency programs - subspecialty certificates, other":"Medical School",
"Energy, environment and natural resources law (llm, ms, msc, lld, jsd/sjd)":"Law"}, inplace=True)
program

,University,Program,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,Environmental Studies,-,-,-,-,-,-,-,-,-,-,NaN,-,12,-,28,-
1,Algoma,Computer science,143,-,195,-,160,-,137,-,120,-,100,-,79,-,260,-
2,Algoma,Engineering,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-
3,Algoma,Engineering,-,-,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-
4,Algoma,Linguistics,20,-,15,-,10,-,NaN,-,10,-,NaN,-,NaN,-,11,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,York,Business,706,23,732,31,699,29,707,28,817,31,727,37,711,33,631,34
2406,York,Business,45,-,43,-,55,-,52,-,43,-,78,-,85,-,121,-
2407,York,History,924,116,784,95,650,94,592,85,512,81,472,74,431,86,441,71
2408,York,Linguistics,573,26,597,22,576,26,571,20,498,19,503,19,506,13,481,NaN


In [15]:
# Replace missing numbers with 5, which is the median value for overall headcounts, and replace - with 0
program = program[program['Program'].notna()]
program.fillna(5,inplace=True)
program.replace(('-'),(0), inplace=True)
program[["2012-2013 Undergraduate", "2012-2013 Graduate","2013-2014 Undergraduate","2013-2014 Graduate","2014-2015 Undergraduate","2014-2015 Graduate","2015-2016 Undergraduate","2015-2016 Graduate","2016-2017 Undergraduate","2016-2017 Graduate","2017-2018 Undergraduate","2017-2018 Graduate","2018-2019 Undergraduate","2018-2019 Graduate","2019-2020 Undergraduate","2019-2020 Graduate"]] = program[["2012-2013 Undergraduate", "2012-2013 Graduate","2013-2014 Undergraduate","2013-2014 Graduate","2014-2015 Undergraduate","2014-2015 Graduate","2015-2016 Undergraduate","2015-2016 Graduate","2016-2017 Undergraduate","2016-2017 Graduate","2017-2018 Undergraduate","2017-2018 Graduate","2018-2019 Undergraduate","2018-2019 Graduate","2019-2020 Undergraduate","2019-2020 Graduate"]].apply(pd.to_numeric)
program

,University,Program,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,Environmental Studies,0,0,0,0,0,0,0,0,0,0,5,0,12,0,28,0
1,Algoma,Computer science,143,0,195,0,160,0,137,0,120,0,100,0,79,0,260,0
2,Algoma,Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0
3,Algoma,Engineering,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0
4,Algoma,Linguistics,20,0,15,0,10,0,5,0,10,0,5,0,5,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,York,Business,706,23,732,31,699,29,707,28,817,31,727,37,711,33,631,34
2406,York,Business,45,0,43,0,55,0,52,0,43,0,78,0,85,0,121,0
2407,York,History,924,116,784,95,650,94,592,85,512,81,472,74,431,86,441,71
2408,York,Linguistics,573,26,597,22,576,26,571,20,498,19,503,19,506,13,481,5


In [16]:
# Group the data by University and Program just in case there were 2 rows with the same names
program = program.groupby(['University','Program']).sum().reset_index()
program

,University,Program,2012-2013 Undergraduate,2012-2013 Graduate,2013-2014 Undergraduate,2013-2014 Graduate,2014-2015 Undergraduate,2014-2015 Graduate,2015-2016 Undergraduate,2015-2016 Graduate,2016-2017 Undergraduate,2016-2017 Graduate,2017-2018 Undergraduate,2017-2018 Graduate,2018-2019 Undergraduate,2018-2019 Graduate,2019-2020 Undergraduate,2019-2020 Graduate
0,Algoma,Arts,44,0,47,0,38,0,41,0,36,0,38,0,39,0,20,0
1,Algoma,Biologies,56,0,68,0,83,0,81,0,77,0,60,0,66,0,98,0
2,Algoma,Business,310,0,359,0,395,0,332,0,297,0,242,0,294,0,685,0
3,Algoma,Computer science,143,0,195,0,160,0,137,0,120,0,100,0,79,0,260,0
4,Algoma,Economics,5,0,5,0,10,0,15,0,10,0,5,0,12,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,York,Psychologies,749,0,702,0,757,0,783,0,856,0,882,0,1106,0,1161,0
417,York,Sciences,740,177,709,168,748,167,668,160,629,159,564,147,541,152,565,160
418,York,Social Studies,5603,502,5487,505,5187,504,5049,516,4866,498,4659,495,4492,521,4283,483
419,York,Statistics,390,87,409,86,445,79,454,92,475,93,545,103,599,114,663,102


In [17]:
#Data validation 1. Ensure no duplicate rows. The first row should always have a 1 in "count". This will ensure that there is only one combination of University and Program
test=program.groupby(['University','Program'],as_index=False).agg(
    count=('Program','count'))
test = test.sort_values(['count'], ascending = (False))
test

,University,Program,count
0,Algoma,Arts,1
264,Toronto,History,1
288,Trent,Psychologies,1
287,Trent,Philosophy,1
286,Trent,Linguistics,1
...,...,...,...
137,McMaster,Computer science,1
136,McMaster,Communication Studies,1
135,McMaster,Business,1
134,McMaster,Biologies,1


In [18]:
#Data validation 2. Ensure there are not programs outside the list of programs we manually created. If there is, they will need to be manually mapped
test=program.groupby(['Program'],as_index=False).agg(
    count=('Program','count'))
test = test.sort_values(['count'], ascending = (False))
test

,Program,count
10,General studies,23
16,Linguistics,20
2,Biologies,20
3,Business,20
21,Social Studies,20
18,Philosophy,20
22,Statistics,19
5,Computer science,19
14,History,19
13,Healthcare Studies,19


In [19]:
# Ensure there are no missing numbers
program.isnull().sum() 

University                 0
Program                    0
2012-2013 Undergraduate    0
2012-2013 Graduate         0
2013-2014 Undergraduate    0
2013-2014 Graduate         0
2014-2015 Undergraduate    0
2014-2015 Graduate         0
2015-2016 Undergraduate    0
2015-2016 Graduate         0
2016-2017 Undergraduate    0
2016-2017 Graduate         0
2017-2018 Undergraduate    0
2017-2018 Graduate         0
2018-2019 Undergraduate    0
2018-2019 Graduate         0
2019-2020 Undergraduate    0
2019-2020 Graduate         0
dtype: int64

In [20]:
# Ensure the data type is correct
program.dtypes

University                 object
Program                    object
2012-2013 Undergraduate     int64
2012-2013 Graduate          int64
2013-2014 Undergraduate     int64
2013-2014 Graduate          int64
2014-2015 Undergraduate     int64
2014-2015 Graduate          int64
2015-2016 Undergraduate     int64
2015-2016 Graduate          int64
2016-2017 Undergraduate     int64
2016-2017 Graduate          int64
2017-2018 Undergraduate     int64
2017-2018 Graduate          int64
2018-2019 Undergraduate     int64
2018-2019 Graduate          int64
2019-2020 Undergraduate     int64
2019-2020 Graduate          int64
dtype: object

##Separating Graduate and Undergraduate Information

Graduate and Undergraduate data needs to be worked on separately as the number of headcounts are dependent on the type of study

In [21]:
#Create another dataframe that isolates graduate headcounts for country
country_grad = country[['University','Country','2012-2013 Graduate', '2013-2014 Graduate', '2014-2015 Graduate', '2015-2016 Graduate', '2016-2017 Graduate', '2017-2018 Graduate', '2018-2019 Graduate', '2019-2020 Graduate']]
country_grad['Total']= country_grad['2012-2013 Graduate']+country_grad['2013-2014 Graduate']+country_grad['2014-2015 Graduate']+country_grad['2015-2016 Graduate']+country_grad['2016-2017 Graduate']+country_grad['2017-2018 Graduate']+country_grad['2018-2019 Graduate']+country_grad['2019-2020 Graduate']
country_grad=country_grad.drop(country_grad[country_grad.Total==0].index)
country_grad=country_grad.drop('Total',1)
country_grad = country_grad[(country_grad[['2012-2013 Graduate', '2013-2014 Graduate', '2014-2015 Graduate', '2015-2016 Graduate', '2016-2017 Graduate', '2017-2018 Graduate', '2018-2019 Graduate', '2019-2020 Graduate']] != 0).all(axis=1)]
country_grad

,University,Country,2012-2013 Graduate,2013-2014 Graduate,2014-2015 Graduate,2015-2016 Graduate,2016-2017 Graduate,2017-2018 Graduate,2018-2019 Graduate,2019-2020 Graduate
99,Brock,Bangladesh,5,5,5,5,11,16,18,32
113,Brock,Canada,571,634,626,660,692,695,740,792
117,Brock,China,370,424,423,387,388,370,350,321
118,Brock,Colombia,5,5,5,5,5,5,5,5
137,Brock,Germany,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...
3013,York,Ukraine,11,5,5,5,5,10,5,5
3015,York,United Kingdom,19,25,21,26,18,13,16,23
3016,York,United States,73,71,72,69,71,66,70,71
3019,York,"Venezuela, Bolivarian Republic of",5,5,5,5,5,5,5,5


In [22]:
#Create another dataframe that isolates graduate headcounts for program
program_grad = program[['University','Program','2012-2013 Graduate', '2013-2014 Graduate', '2014-2015 Graduate', '2015-2016 Graduate', '2016-2017 Graduate', '2017-2018 Graduate', '2018-2019 Graduate', '2019-2020 Graduate']]
program_grad['Total']= program_grad['2012-2013 Graduate']+program_grad['2013-2014 Graduate']+program_grad['2014-2015 Graduate']+program_grad['2015-2016 Graduate']+program_grad['2016-2017 Graduate']+program_grad['2017-2018 Graduate']+program_grad['2018-2019 Graduate']+program_grad['2019-2020 Graduate']
program_grad=program_grad.drop(program_grad[program_grad.Total==0].index)
program_grad=program_grad.drop('Total',1)
program_grad = program_grad[(program_grad[['2012-2013 Graduate', '2013-2014 Graduate', '2014-2015 Graduate', '2015-2016 Graduate', '2016-2017 Graduate', '2017-2018 Graduate', '2018-2019 Graduate', '2019-2020 Graduate']] != 0).all(axis=1)]
program_grad

,University,Program,2012-2013 Graduate,2013-2014 Graduate,2014-2015 Graduate,2015-2016 Graduate,2016-2017 Graduate,2017-2018 Graduate,2018-2019 Graduate,2019-2020 Graduate
16,Brock,Biologies,53,57,52,54,52,54,54,65
17,Brock,Business,487,553,501,475,533,530,531,598
19,Brock,Computer science,18,16,12,13,14,19,29,27
23,Brock,General studies,5,11,12,14,17,5,5,5
24,Brock,Geographies,12,17,15,15,15,17,11,14
...,...,...,...,...,...,...,...,...,...,...
415,York,Philosophy,252,247,236,236,232,231,236,240
417,York,Sciences,177,168,167,160,159,147,152,160
418,York,Social Studies,502,505,504,516,498,495,521,483
419,York,Statistics,87,86,79,92,93,103,114,102


In [23]:
#Create another dataframe that isolates undergraduate headcounts for country
country_undergrad = country[['University','Country','2012-2013 Undergraduate', '2013-2014 Undergraduate', '2014-2015 Undergraduate', '2015-2016 Undergraduate', '2016-2017 Undergraduate', '2017-2018 Undergraduate', '2018-2019 Undergraduate', '2019-2020 Undergraduate']]
country_undergrad['Total']= country_undergrad['2012-2013 Undergraduate']+country_undergrad['2013-2014 Undergraduate']+country_undergrad['2014-2015 Undergraduate']+country_undergrad['2015-2016 Undergraduate']+country_undergrad['2016-2017 Undergraduate']+country_undergrad['2017-2018 Undergraduate']+country_undergrad['2018-2019 Undergraduate']+country_undergrad['2019-2020 Undergraduate']
country_undergrad=country_undergrad.drop(country_undergrad[country_undergrad.Total==0].index)
country_undergrad=country_undergrad.drop('Total',1)
country_undergrad = country_undergrad[(country_undergrad[['2012-2013 Undergraduate', '2013-2014 Undergraduate', '2014-2015 Undergraduate', '2015-2016 Undergraduate', '2016-2017 Undergraduate', '2017-2018 Undergraduate', '2018-2019 Undergraduate', '2019-2020 Undergraduate']] != 0).all(axis=1)]
country_undergrad

,University,Country,2012-2013 Undergraduate,2013-2014 Undergraduate,2014-2015 Undergraduate,2015-2016 Undergraduate,2016-2017 Undergraduate,2017-2018 Undergraduate,2018-2019 Undergraduate,2019-2020 Undergraduate
6,Algoma,Bangladesh,5,5,5,5,5,5,12,5
12,Algoma,Canada,871,861,826,732,714,658,677,674
13,Algoma,China,38,46,44,40,35,34,32,33
26,Algoma,Germany,5,5,5,5,5,5,5,5
27,Algoma,Ghana,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...
3019,York,"Venezuela, Bolivarian Republic of",33,36,39,33,24,22,22,28
3020,York,Viet Nam,129,113,112,131,116,132,181,221
3022,York,Yemen,5,5,11,5,5,5,17,26
3023,York,Zambia,5,5,5,5,5,5,5,5


In [24]:
#Create another dataframe that isolates undergraduate headcounts for program
program_undergrad = program[['University','Program','2012-2013 Undergraduate', '2013-2014 Undergraduate', '2014-2015 Undergraduate', '2015-2016 Undergraduate', '2016-2017 Undergraduate', '2017-2018 Undergraduate', '2018-2019 Undergraduate', '2019-2020 Undergraduate']]
program_undergrad['Total']= program_undergrad['2012-2013 Undergraduate']+program_undergrad['2013-2014 Undergraduate']+program_undergrad['2014-2015 Undergraduate']+program_undergrad['2015-2016 Undergraduate']+program_undergrad['2016-2017 Undergraduate']+program_undergrad['2017-2018 Undergraduate']+program_undergrad['2018-2019 Undergraduate']+program_undergrad['2019-2020 Undergraduate']
program_undergrad=program_undergrad.drop(program_undergrad[program_undergrad.Total==0].index)
program_undergrad=program_undergrad.drop('Total',1)
program_undergrad = program_undergrad[(program_undergrad[['2012-2013 Undergraduate', '2013-2014 Undergraduate', '2014-2015 Undergraduate', '2015-2016 Undergraduate', '2016-2017 Undergraduate', '2017-2018 Undergraduate', '2018-2019 Undergraduate', '2019-2020 Undergraduate']] != 0).all(axis=1)]
program_undergrad

,University,Program,2012-2013 Undergraduate,2013-2014 Undergraduate,2014-2015 Undergraduate,2015-2016 Undergraduate,2016-2017 Undergraduate,2017-2018 Undergraduate,2018-2019 Undergraduate,2019-2020 Undergraduate
0,Algoma,Arts,44,47,38,41,36,38,39,20
1,Algoma,Biologies,56,68,83,81,77,60,66,98
2,Algoma,Business,310,359,395,332,297,242,294,685
3,Algoma,Computer science,143,195,160,137,120,100,79,260
4,Algoma,Economics,5,5,10,15,10,5,12,5
...,...,...,...,...,...,...,...,...,...,...
416,York,Psychologies,749,702,757,783,856,882,1106,1161
417,York,Sciences,740,709,748,668,629,564,541,565
418,York,Social Studies,5603,5487,5187,5049,4866,4659,4492,4283
419,York,Statistics,390,409,445,454,475,545,599,663


##Merging Data Sets Country and Program

The datasets are separated by Country and Program. By merging, an approximate value in the chain of University>Country>Program will be created

In [25]:
#Perform an outer join of the two graduate dataframes
acp_grad = pd.merge(program_grad, country_grad, how="outer", on="University")
acp_grad = acp_grad[['University','Program','Country','2012-2013 Graduate_y','2012-2013 Graduate_x','2013-2014 Graduate_y','2013-2014 Graduate_x','2014-2015 Graduate_y','2014-2015 Graduate_x','2015-2016 Graduate_y','2015-2016 Graduate_x','2016-2017 Graduate_y','2016-2017 Graduate_x','2017-2018 Graduate_y','2017-2018 Graduate_x','2018-2019 Graduate_y','2018-2019 Graduate_x','2019-2020 Graduate_y','2019-2020 Graduate_x']]
acp_grad

,University,Program,Country,2012-2013 Graduate_y,2012-2013 Graduate_x,2013-2014 Graduate_y,2013-2014 Graduate_x,2014-2015 Graduate_y,2014-2015 Graduate_x,2015-2016 Graduate_y,2015-2016 Graduate_x,2016-2017 Graduate_y,2016-2017 Graduate_x,2017-2018 Graduate_y,2017-2018 Graduate_x,2018-2019 Graduate_y,2018-2019 Graduate_x,2019-2020 Graduate_y,2019-2020 Graduate_x
0,Brock,Biologies,Bangladesh,5,53,5,57,5,52,5,54,11,52,16,54,18,54,32,65
1,Brock,Biologies,Canada,571,53,634,57,626,52,660,54,692,52,695,54,740,54,792,65
2,Brock,Biologies,China,370,53,424,57,423,52,387,54,388,52,370,54,350,54,321,65
3,Brock,Biologies,Colombia,5,53,5,57,5,52,5,54,5,52,5,54,5,54,5,65
4,Brock,Biologies,Germany,5,53,5,57,5,52,5,54,5,52,5,54,5,54,5,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14073,York,Teacher Education,Ukraine,11,136,5,131,5,145,5,145,5,164,10,190,5,196,5,175
14074,York,Teacher Education,United Kingdom,19,136,25,131,21,145,26,145,18,164,13,190,16,196,23,175
14075,York,Teacher Education,United States,73,136,71,131,72,145,69,145,71,164,66,190,70,196,71,175
14076,York,Teacher Education,"Venezuela, Bolivarian Republic of",5,136,5,131,5,145,5,145,5,164,5,190,5,196,5,175


In [26]:
#Sum graduate headcount by university
totals=acp_grad.groupby(['University','Program'],as_index=False).agg(
    sum1=('2012-2013 Graduate_y','sum'),sum2=('2013-2014 Graduate_y','sum'),sum3=('2014-2015 Graduate_y','sum'),sum4=('2015-2016 Graduate_y','sum'),sum5=('2016-2017 Graduate_y','sum'),sum6=('2017-2018 Graduate_y','sum'),sum7=('2018-2019 Graduate_y','sum'),sum8=('2019-2020 Graduate_y','sum'))
del totals['Program']
totals.drop_duplicates(keep='first',inplace=True)
totals

,University,sum1,sum2,sum3,sum4,sum5,sum6,sum7,sum8
0,Brock,1177,1280,1228,1225,1286,1309,1376,1528
15,Carleton,2950,3034,3114,3301,3462,3571,3472,3617
34,Guelph,2388,2324,2274,2277,2489,2616,2680,2821
53,Lakehead,745,789,789,873,1000,1174,1260,1409
67,Laurentian,394,450,462,486,484,472,541,596
78,McMaster,3481,3470,3574,3573,3756,3858,4163,4323
96,Nipissing,90,99,132,156,140,148,161,152
99,O.C.A.D,127,137,140,150,157,161,179,186
100,Ottawa,5189,5481,5658,5595,5510,5721,5983,6308
121,Queen's,3759,3819,3956,4070,4092,4293,4582,4755


In [27]:
#Join the sum with the graduate dataframe
acp_grad2 = pd.merge(acp_grad, totals, how="left", on="University")
acp_grad2

,University,Program,Country,2012-2013 Graduate_y,2012-2013 Graduate_x,2013-2014 Graduate_y,2013-2014 Graduate_x,2014-2015 Graduate_y,2014-2015 Graduate_x,2015-2016 Graduate_y,...,2019-2020 Graduate_y,2019-2020 Graduate_x,sum1,sum2,sum3,sum4,sum5,sum6,sum7,sum8
0,Brock,Biologies,Bangladesh,5,53,5,57,5,52,5,...,32,65,1177,1280,1228,1225,1286,1309,1376,1528
1,Brock,Biologies,Canada,571,53,634,57,626,52,660,...,792,65,1177,1280,1228,1225,1286,1309,1376,1528
2,Brock,Biologies,China,370,53,424,57,423,52,387,...,321,65,1177,1280,1228,1225,1286,1309,1376,1528
3,Brock,Biologies,Colombia,5,53,5,57,5,52,5,...,5,65,1177,1280,1228,1225,1286,1309,1376,1528
4,Brock,Biologies,Germany,5,53,5,57,5,52,5,...,5,65,1177,1280,1228,1225,1286,1309,1376,1528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14073,York,Teacher Education,Ukraine,11,136,5,131,5,145,5,...,5,175,3887,3897,3978,4022,4061,4174,4361,4429
14074,York,Teacher Education,United Kingdom,19,136,25,131,21,145,26,...,23,175,3887,3897,3978,4022,4061,4174,4361,4429
14075,York,Teacher Education,United States,73,136,71,131,72,145,69,...,71,175,3887,3897,3978,4022,4061,4174,4361,4429
14076,York,Teacher Education,"Venezuela, Bolivarian Republic of",5,136,5,131,5,145,5,...,5,175,3887,3897,3978,4022,4061,4174,4361,4429


In [28]:
#Use the graduate dataframe and university sum to estimate the headcount per program and country and delete all rows with 0 headcounts for all years
acp_grad2['2012'] = acp_grad2['2012-2013 Graduate_y']/acp_grad2['sum1']*acp_grad2['2012-2013 Graduate_x']
acp_grad2['2013'] = acp_grad2['2013-2014 Graduate_y']/acp_grad2['sum2']*acp_grad2['2013-2014 Graduate_x']
acp_grad2['2014'] = acp_grad2['2014-2015 Graduate_y']/acp_grad2['sum3']*acp_grad2['2014-2015 Graduate_x']
acp_grad2['2015'] = acp_grad2['2015-2016 Graduate_y']/acp_grad2['sum4']*acp_grad2['2015-2016 Graduate_x']
acp_grad2['2016'] = acp_grad2['2016-2017 Graduate_y']/acp_grad2['sum5']*acp_grad2['2016-2017 Graduate_x']
acp_grad2['2017'] = acp_grad2['2017-2018 Graduate_y']/acp_grad2['sum6']*acp_grad2['2017-2018 Graduate_x']
acp_grad2['2018'] = acp_grad2['2018-2019 Graduate_y']/acp_grad2['sum7']*acp_grad2['2018-2019 Graduate_x']
acp_grad2['2019'] = acp_grad2['2019-2020 Graduate_y']/acp_grad2['sum8']*acp_grad2['2019-2020 Graduate_x']
acp_grad2.fillna(0,inplace=True)
acp_grad2[['2012','2013','2014','2015','2016','2017','2018','2019']] = round(acp_grad2[['2012','2013','2014','2015','2016','2017','2018','2019']])
acp_grad2[['2012','2013','2014','2015','2016','2017','2018','2019']] = acp_grad2[['2012','2013','2014','2015','2016','2017','2018','2019']].astype(int)
acp_grad2['Total']= acp_grad2['2012']+acp_grad2['2013']+acp_grad2['2014']+acp_grad2['2015']+acp_grad2['2016']+acp_grad2['2017']+acp_grad2['2018']+acp_grad2['2019']
acp_grad2=acp_grad2.drop(acp_grad2[acp_grad2.Total==0].index)
acp_grad2

,University,Program,Country,2012-2013 Graduate_y,2012-2013 Graduate_x,2013-2014 Graduate_y,2013-2014 Graduate_x,2014-2015 Graduate_y,2014-2015 Graduate_x,2015-2016 Graduate_y,...,sum8,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,Brock,Biologies,Bangladesh,5,53,5,57,5,52,5,...,1528,0,0,0,0,0,1,1,1,3
1,Brock,Biologies,Canada,571,53,634,57,626,52,660,...,1528,26,28,27,29,28,29,29,34,230
2,Brock,Biologies,China,370,53,424,57,423,52,387,...,1528,17,19,18,17,16,15,14,14,130
5,Brock,Biologies,Ghana,18,53,26,57,24,52,17,...,1528,1,1,1,1,1,1,1,1,8
7,Brock,Biologies,India,55,53,56,57,44,52,45,...,1528,2,2,2,2,2,3,5,10,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14063,York,Teacher Education,Saudi Arabia,16,136,16,131,20,145,22,...,4429,1,1,1,1,1,1,1,0,7
14071,York,Teacher Education,Turkey,19,136,21,131,25,145,20,...,4429,1,1,1,1,1,1,1,1,8
14074,York,Teacher Education,United Kingdom,19,136,25,131,21,145,26,...,4429,1,1,1,1,1,1,1,1,8
14075,York,Teacher Education,United States,73,136,71,131,72,145,69,...,4429,3,2,3,2,3,3,3,3,22


In [29]:
#Isolate the dataframe to only headcounts
acp_grad_final = acp_grad2[['University','Country','Program','2012','2013','2014','2015','2016','2017','2018','2019']]
acp_grad_final = acp_grad_final.reset_index(drop=True)
acp_grad_final

,University,Country,Program,2012,2013,2014,2015,2016,2017,2018,2019
0,Brock,Bangladesh,Biologies,0,0,0,0,0,1,1,1
1,Brock,Canada,Biologies,26,28,27,29,28,29,29,34
2,Brock,China,Biologies,17,19,18,17,16,15,14,14
3,Brock,Ghana,Biologies,1,1,1,1,1,1,1,1
4,Brock,India,Biologies,2,2,2,2,2,3,5,10
...,...,...,...,...,...,...,...,...,...,...,...
6137,York,Saudi Arabia,Teacher Education,1,1,1,1,1,1,1,0
6138,York,Turkey,Teacher Education,1,1,1,1,1,1,1,1
6139,York,United Kingdom,Teacher Education,1,1,1,1,1,1,1,1
6140,York,United States,Teacher Education,3,2,3,2,3,3,3,3


In [30]:
#Perform an outer join of the two undergraduate dataframes
acp_undergrad = pd.merge(program_undergrad, country_undergrad, how="outer", on="University")
acp_undergrad = acp_undergrad[['University','Program','Country','2012-2013 Undergraduate_y','2012-2013 Undergraduate_x','2013-2014 Undergraduate_y','2013-2014 Undergraduate_x','2014-2015 Undergraduate_y','2014-2015 Undergraduate_x','2015-2016 Undergraduate_y','2015-2016 Undergraduate_x','2016-2017 Undergraduate_y','2016-2017 Undergraduate_x','2017-2018 Undergraduate_y','2017-2018 Undergraduate_x','2018-2019 Undergraduate_y','2018-2019 Undergraduate_x','2019-2020 Undergraduate_y','2019-2020 Undergraduate_x']]
acp_undergrad

,University,Program,Country,2012-2013 Undergraduate_y,2012-2013 Undergraduate_x,2013-2014 Undergraduate_y,2013-2014 Undergraduate_x,2014-2015 Undergraduate_y,2014-2015 Undergraduate_x,2015-2016 Undergraduate_y,2015-2016 Undergraduate_x,2016-2017 Undergraduate_y,2016-2017 Undergraduate_x,2017-2018 Undergraduate_y,2017-2018 Undergraduate_x,2018-2019 Undergraduate_y,2018-2019 Undergraduate_x,2019-2020 Undergraduate_y,2019-2020 Undergraduate_x
0,Algoma,Arts,Bangladesh,5,44,5,47,5,38,5,41,5,36,5,38,12,39,5,20
1,Algoma,Arts,Canada,871,44,861,47,826,38,732,41,714,36,658,38,677,39,674,20
2,Algoma,Arts,China,38,44,46,47,44,38,40,41,35,36,34,38,32,39,33,20
3,Algoma,Arts,Germany,5,44,5,47,5,38,5,41,5,36,5,38,5,39,5,20
4,Algoma,Arts,Ghana,5,44,5,47,5,38,5,41,5,36,5,38,5,39,5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28864,York,Teacher Education,"Venezuela, Bolivarian Republic of",33,538,36,537,39,551,33,324,24,845,22,1202,22,1436,28,1600
28865,York,Teacher Education,Viet Nam,129,538,113,537,112,551,131,324,116,845,132,1202,181,1436,221,1600
28866,York,Teacher Education,Yemen,5,538,5,537,11,551,5,324,5,845,5,1202,17,1436,26,1600
28867,York,Teacher Education,Zambia,5,538,5,537,5,551,5,324,5,845,5,1202,5,1436,5,1600


In [31]:
#Sum undergraduate headcount by university
totals=acp_undergrad.groupby(['University','Program'],as_index=False).agg(
    sum1=('2012-2013 Undergraduate_y','sum'),sum2=('2013-2014 Undergraduate_y','sum'),sum3=('2014-2015 Undergraduate_y','sum'),sum4=('2015-2016 Undergraduate_y','sum'),sum5=('2016-2017 Undergraduate_y','sum'),sum6=('2017-2018 Undergraduate_y','sum'),sum7=('2018-2019 Undergraduate_y','sum'),sum8=('2019-2020 Undergraduate_y','sum'))
del totals['Program']
totals.drop_duplicates(keep='first',inplace=True)
totals

,University,sum1,sum2,sum3,sum4,sum5,sum6,sum7,sum8
0,Algoma,1081,1176,1162,1041,954,849,904,1517
12,Brock,14645,14826,14878,14611,14777,14711,14853,15391
32,Carleton,19378,19978,20348,20565,20943,21693,21737,21762
54,Guelph,22502,22675,22918,23681,24245,24149,24064,24375
75,Hearst,69,67,69,90,63,79,82,76
78,Lakehead,6333,6258,6161,5657,5786,5557,5791,5761
99,Laurentian,6251,6262,6270,6562,6416,6208,6061,6073
121,McMaster,22200,22568,22829,24759,25965,26600,27647,28725
142,NOSM,378,408,420,427,433,435,446,446
144,Nipissing,3660,3399,3267,3097,3077,3011,3357,3550


In [32]:
#Join the sum with the undergraduate dataframe
acp_undergrad2 = pd.merge(acp_undergrad, totals, how="left", on="University")
acp_undergrad2

,University,Program,Country,2012-2013 Undergraduate_y,2012-2013 Undergraduate_x,2013-2014 Undergraduate_y,2013-2014 Undergraduate_x,2014-2015 Undergraduate_y,2014-2015 Undergraduate_x,2015-2016 Undergraduate_y,...,2019-2020 Undergraduate_y,2019-2020 Undergraduate_x,sum1,sum2,sum3,sum4,sum5,sum6,sum7,sum8
0,Algoma,Arts,Bangladesh,5,44,5,47,5,38,5,...,5,20,1081,1176,1162,1041,954,849,904,1517
1,Algoma,Arts,Canada,871,44,861,47,826,38,732,...,674,20,1081,1176,1162,1041,954,849,904,1517
2,Algoma,Arts,China,38,44,46,47,44,38,40,...,33,20,1081,1176,1162,1041,954,849,904,1517
3,Algoma,Arts,Germany,5,44,5,47,5,38,5,...,5,20,1081,1176,1162,1041,954,849,904,1517
4,Algoma,Arts,Ghana,5,44,5,47,5,38,5,...,5,20,1081,1176,1162,1041,954,849,904,1517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28864,York,Teacher Education,"Venezuela, Bolivarian Republic of",33,538,36,537,39,551,33,...,28,1600,41240,40901,39675,39598,39636,41207,42925,42988
28865,York,Teacher Education,Viet Nam,129,538,113,537,112,551,131,...,221,1600,41240,40901,39675,39598,39636,41207,42925,42988
28866,York,Teacher Education,Yemen,5,538,5,537,11,551,5,...,26,1600,41240,40901,39675,39598,39636,41207,42925,42988
28867,York,Teacher Education,Zambia,5,538,5,537,5,551,5,...,5,1600,41240,40901,39675,39598,39636,41207,42925,42988


In [33]:
#Use the undergraduate dataframe and university sum to estimate the headcount per program and country and delete all rows with 0 headcounts for all years
acp_undergrad2['2012'] = acp_undergrad2['2012-2013 Undergraduate_y']/acp_undergrad2['sum1']*acp_undergrad2['2012-2013 Undergraduate_x']
acp_undergrad2['2013'] = acp_undergrad2['2013-2014 Undergraduate_y']/acp_undergrad2['sum2']*acp_undergrad2['2013-2014 Undergraduate_x']
acp_undergrad2['2014'] = acp_undergrad2['2014-2015 Undergraduate_y']/acp_undergrad2['sum3']*acp_undergrad2['2014-2015 Undergraduate_x']
acp_undergrad2['2015'] = acp_undergrad2['2015-2016 Undergraduate_y']/acp_undergrad2['sum4']*acp_undergrad2['2015-2016 Undergraduate_x']
acp_undergrad2['2016'] = acp_undergrad2['2016-2017 Undergraduate_y']/acp_undergrad2['sum5']*acp_undergrad2['2016-2017 Undergraduate_x']
acp_undergrad2['2017'] = acp_undergrad2['2017-2018 Undergraduate_y']/acp_undergrad2['sum6']*acp_undergrad2['2017-2018 Undergraduate_x']
acp_undergrad2['2018'] = acp_undergrad2['2018-2019 Undergraduate_y']/acp_undergrad2['sum7']*acp_undergrad2['2018-2019 Undergraduate_x']
acp_undergrad2['2019'] = acp_undergrad2['2019-2020 Undergraduate_y']/acp_undergrad2['sum8']*acp_undergrad2['2019-2020 Undergraduate_x']
acp_undergrad2.fillna(0,inplace=True)
acp_undergrad2[['2012','2013','2014','2015','2016','2017','2018','2019']] = round(acp_undergrad2[['2012','2013','2014','2015','2016','2017','2018','2019']])
acp_undergrad2[['2012','2013','2014','2015','2016','2017','2018','2019']] = acp_undergrad2[['2012','2013','2014','2015','2016','2017','2018','2019']].astype(int)
acp_undergrad2['Total']= acp_undergrad2['2012']+acp_undergrad2['2013']+acp_undergrad2['2014']+acp_undergrad2['2015']+acp_undergrad2['2016']+acp_undergrad2['2017']+acp_undergrad2['2018']+acp_undergrad2['2019']
acp_undergrad2=acp_undergrad2.drop(acp_undergrad2[acp_undergrad2.Total==0].index)
acp_undergrad2

,University,Program,Country,2012-2013 Undergraduate_y,2012-2013 Undergraduate_x,2013-2014 Undergraduate_y,2013-2014 Undergraduate_x,2014-2015 Undergraduate_y,2014-2015 Undergraduate_x,2015-2016 Undergraduate_y,...,sum8,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,Algoma,Arts,Bangladesh,5,44,5,47,5,38,5,...,1517,0,0,0,0,0,0,1,0,1
1,Algoma,Arts,Canada,871,44,861,47,826,38,732,...,1517,35,34,27,29,27,29,29,9,219
2,Algoma,Arts,China,38,44,46,47,44,38,40,...,1517,2,2,1,2,1,2,1,0,11
5,Algoma,Arts,India,10,44,5,47,12,38,10,...,1517,0,0,0,0,0,0,5,10,15
6,Algoma,Arts,Japan,5,44,5,47,5,38,5,...,1517,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28861,York,Teacher Education,United States,161,538,143,537,157,551,154,...,42988,2,2,2,1,3,5,6,7,28
28864,York,Teacher Education,"Venezuela, Bolivarian Republic of",33,538,36,537,39,551,33,...,42988,0,0,1,0,1,1,1,1,5
28865,York,Teacher Education,Viet Nam,129,538,113,537,112,551,131,...,42988,2,1,2,1,2,4,6,8,26
28866,York,Teacher Education,Yemen,5,538,5,537,11,551,5,...,42988,0,0,0,0,0,0,1,1,2


In [34]:
#Isolate the dataframe to only headcounts
acp_undergrad_final = acp_undergrad2[['University','Country','Program','2012','2013','2014','2015','2016','2017','2018','2019']]
acp_undergrad_final = acp_undergrad_final.reset_index(drop=True)
acp_undergrad_final

,University,Country,Program,2012,2013,2014,2015,2016,2017,2018,2019
0,Algoma,Bangladesh,Arts,0,0,0,0,0,0,1,0
1,Algoma,Canada,Arts,35,34,27,29,27,29,29,9
2,Algoma,China,Arts,2,2,1,2,1,2,1,0
3,Algoma,India,Arts,0,0,0,0,0,0,5,10
4,Algoma,Japan,Arts,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14380,York,United States,Teacher Education,2,2,2,1,3,5,6,7
14381,York,"Venezuela, Bolivarian Republic of",Teacher Education,0,0,1,0,1,1,1,1
14382,York,Viet Nam,Teacher Education,2,1,2,1,2,4,6,8
14383,York,Yemen,Teacher Education,0,0,0,0,0,0,1,1


In [35]:
#Ensure no duplicate rows
test=acp_undergrad_final.groupby(['University','Country','Program'],as_index=False).agg(
    count=('2017','count'))
test = test.sort_values(['count'], ascending = (False))
test

,University,Country,Program,count
0,Algoma,Bangladesh,Arts,1
9665,UOIT,Peru,Engineering,1
9583,UOIT,Iraq,Psychologies,1
9584,UOIT,Iraq,Sciences,1
9585,UOIT,Iraq,Teacher Education,1
...,...,...,...,...
4799,Ottawa,Morocco,Business,1
4800,Ottawa,Morocco,Communication Studies,1
4801,Ottawa,Morocco,Computer science,1
4802,Ottawa,Morocco,Cultural Studies,1


#Master Dataset

##Model Data Set Graduate

In [36]:
#Melt all the 8 columns for the years in one column only and naming it "Year". Sort values and transform the column as a Datetime
model_acp = acp_grad_final.melt(id_vars=["University", "Country","Program"], 
    var_name="Year", 
    value_name="Headcount")


print(model_acp.shape)
model_acp = model_acp.sort_values(['University', 'Country','Program','Year'], ascending = (True, True, True, True))
model_acp['Year'] = pd.to_datetime(model_acp['Year'])
model_acp

(49136, 5)


,University,Country,Program,Year,Headcount
0,Brock,Bangladesh,Biologies,2012-01-01,0
6142,Brock,Bangladesh,Biologies,2013-01-01,0
12284,Brock,Bangladesh,Biologies,2014-01-01,0
18426,Brock,Bangladesh,Biologies,2015-01-01,0
24568,Brock,Bangladesh,Biologies,2016-01-01,0
...,...,...,...,...,...
24567,York,Viet Nam,Teacher Education,2015-01-01,0
30709,York,Viet Nam,Teacher Education,2016-01-01,1
36851,York,Viet Nam,Teacher Education,2017-01-01,1
42993,York,Viet Nam,Teacher Education,2018-01-01,0


In [37]:
# Create the relevant pivot table used to further analyze the data
model_acp_pivot = model_acp.pivot_table(
    values='Headcount',
    index=['Year'],
    columns=['University','Country','Program'],
    aggfunc='sum'
    )

# Formatting.
model_acp_pivot.columns.name = None
print(model_acp_pivot.shape)
model_acp_pivot = model_acp_pivot.drop(columns=model_acp_pivot.columns[(model_acp_pivot < 1).any()]) #If any combination (Column) has a value that is less than 1, the column will be dropped
print(model_acp_pivot.shape)
model_acp_pivot

(8, 6142)
(8, 3880)


University      Brock                                                   \
Country    Bangladesh                 Canada                             
Program      Business Psychologies Biologies Business Computer science   
Year                                                                     
2012-01-01          2            1        26      236                9   
2013-01-01          2            1        28      274                8   
2014-01-01          2            1        27      255                6   
2015-01-01          2            1        29      256                7   
2016-01-01          5            1        28      287                8   
2017-01-01          6            2        29      281               10   
2018-01-01          7            3        29      286               16   
2019-01-01         13            6        34      310               14   

University                                                                \
Country                                                                    
Program    General studies Geographies Health Science Healthcare Studies   
Year                                                                       
2012-01-01               2           6             44                 22   
2013-01-01               5           8             37                 26   
2014-01-01               6           8             36                 28   
2015-01-01               8           8             47                 25   
2016-01-01               9           8             53                 36   
2017-01-01               3           9             53                 44   
2018-01-01               3           6             52                 53   
2019-01-01               3           7             57                 60   

University          ...          York                                     \
Country             ... United States                                      
Program    History  ...   Linguistics Philosophy Sciences Social Studies   
Year                ...                                                    
2012-01-01      14  ...             3          5        3              9   
2013-01-01      16  ...             4          5        3              9   
2014-01-01      19  ...             4          4        3              9   
2015-01-01      19  ...             3          4        3              9   
2016-01-01      22  ...             3          4        3              9   
2017-01-01      23  ...             3          4        2              8   
2018-01-01      19  ...             3          4        2              8   
2019-01-01      17  ...             3          4        3              8   

University                                                                 \
Country                                 Venezuela, Bolivarian Republic of   
Program    Statistics Teacher Education                          Business   
Year                                                                        
2012-01-01          2                 3                                 1   
2013-01-01          2                 2                                 1   
2014-01-01          1                 3                                 1   
2015-01-01          2                 2                                 1   
2016-01-01          2                 3                                 1   
2017-01-01          2                 3                                 1   
2018-01-01          2                 3                                 1   
2019-01-01          2                 3                                 1   

University                                         
Country                   Viet Nam                 
Program    Social Studies Business Social Studies  
Year                                               
2012-01-01              1        1              1  
2013-01-01              1        1              1  
2014-01-01              1        1              

In [38]:
# Extract the university, country, and program columns used to create the final master set
topfinal= pd.DataFrame()
a= np.array(model_acp_pivot.columns)
for name, column in model_acp_pivot.iteritems(): 
    for i in name: 
        topfinal= topfinal.append({'University':name[0],'Country':name[1],'Program':name[2]},ignore_index=True)
topfinal.drop_duplicates(keep='first',inplace=True)
topfinal= topfinal[['University','Country','Program']].reset_index(drop=True)
topfinal

,University,Country,Program
0,Brock,Bangladesh,Business
1,Brock,Bangladesh,Psychologies
2,Brock,Canada,Biologies
3,Brock,Canada,Business
4,Brock,Canada,Computer science
...,...,...,...
3875,York,United States,Teacher Education
3876,York,"Venezuela, Bolivarian Republic of",Business
3877,York,"Venezuela, Bolivarian Republic of",Social Studies
3878,York,Viet Nam,Business


In [39]:
# Merge the columns with the relevant headcount for each combination of university, country, program, and year, this is done to convert the 2 dimensional table into a 1 dimensional table for analysis purpose
model_acp = pd.merge(model_acp, topfinal, on=['University','Country','Program'], how="outer", indicator=True)
model_acp = model_acp.loc[model_acp["_merge"] == "both"].drop(["_merge"], axis=1)
model_acp = model_acp.reset_index(drop=True)
model_acp

,University,Country,Program,Year,Headcount
0,Brock,Bangladesh,Business,2012-01-01,2
1,Brock,Bangladesh,Business,2013-01-01,2
2,Brock,Bangladesh,Business,2014-01-01,2
3,Brock,Bangladesh,Business,2015-01-01,2
4,Brock,Bangladesh,Business,2016-01-01,5
...,...,...,...,...,...
31035,York,Viet Nam,Social Studies,2015-01-01,1
31036,York,Viet Nam,Social Studies,2016-01-01,2
31037,York,Viet Nam,Social Studies,2017-01-01,1
31038,York,Viet Nam,Social Studies,2018-01-01,1


In [40]:
#Extract the Ontario average headcount for each combination of country, program, and year (Used for visualization in tableau)
totals=model_acp.groupby(['Country','Program','Year'],as_index=False).agg(
    Average_Ontario=('Headcount','mean'))
totals['Average_Ontario'] = round(totals['Average_Ontario'],2)
totals

,Country,Program,Year,Average_Ontario
0,Afghanistan,Business,2012-01-01,1.0
1,Afghanistan,Business,2013-01-01,1.0
2,Afghanistan,Business,2014-01-01,1.0
3,Afghanistan,Business,2015-01-01,1.0
4,Afghanistan,Business,2016-01-01,1.0
...,...,...,...,...
8467,Zimbabwe,Social Studies,2015-01-01,1.0
8468,Zimbabwe,Social Studies,2016-01-01,1.0
8469,Zimbabwe,Social Studies,2017-01-01,1.0
8470,Zimbabwe,Social Studies,2018-01-01,1.0


In [41]:
#Join the Ontario average with the university headcount. Create a new column called "Type" used in Tableau to separate Graduate and Undergraduate studies
model_acp_grad = pd.merge(model_acp, totals, how="left", on=["Country","Program","Year"])
model_acp_grad["Type"] = "Graduate"
model_acp_grad

,University,Country,Program,Year,Headcount,Average_Ontario,Type
0,Brock,Bangladesh,Business,2012-01-01,2,3.77,Graduate
1,Brock,Bangladesh,Business,2013-01-01,2,4.38,Graduate
2,Brock,Bangladesh,Business,2014-01-01,2,4.54,Graduate
3,Brock,Bangladesh,Business,2015-01-01,2,4.31,Graduate
4,Brock,Bangladesh,Business,2016-01-01,5,4.46,Graduate
...,...,...,...,...,...,...,...
31035,York,Viet Nam,Social Studies,2015-01-01,1,1.20,Graduate
31036,York,Viet Nam,Social Studies,2016-01-01,2,1.60,Graduate
31037,York,Viet Nam,Social Studies,2017-01-01,1,1.40,Graduate
31038,York,Viet Nam,Social Studies,2018-01-01,1,1.20,Graduate


##Model Data Set Undergraduate

In [ ]:
#Melt all the 8 columns for the years in one column only and naming it "Year". Sort values and transform the column as a Datetime
model_acp = acp_undergrad_final.melt(id_vars=["University", "Country","Program"], 
    var_name="Year", 
    value_name="Headcount")

print(model_acp.shape)
model_acp = model_acp.sort_values(['University', 'Country','Program','Year'], ascending = (True, True, True, True))
model_acp['Year'] = pd.to_datetime(model_acp['Year'])
model_acp

(115080, 5)


,University,Country,Program,Year,Headcount
0,Algoma,Bangladesh,Arts,2012-01-01,0
14385,Algoma,Bangladesh,Arts,2013-01-01,0
28770,Algoma,Bangladesh,Arts,2014-01-01,0
43155,Algoma,Bangladesh,Arts,2015-01-01,0
57540,Algoma,Bangladesh,Arts,2016-01-01,0
...,...,...,...,...,...
57539,York,Zimbabwe,Teacher Education,2015-01-01,0
71924,York,Zimbabwe,Teacher Education,2016-01-01,0
86309,York,Zimbabwe,Teacher Education,2017-01-01,1
100694,York,Zimbabwe,Teacher Education,2018-01-01,1


In [ ]:
# Create the relevant pivot table used to further analyze the data
model_acp_pivot = model_acp.pivot_table(
    values='Headcount',
    index=['Year'],
    columns=['University','Country','Program'],
    aggfunc='sum'
    )

# Formatting.
model_acp_pivot.columns.name = None
print(model_acp_pivot.shape)
model_acp_pivot = model_acp_pivot.drop(columns=model_acp_pivot.columns[(model_acp_pivot < 1).any()])  #If any combination (Column) has a value that is less than 1, the column will be dropped
print(model_acp_pivot.shape)
model_acp_pivot

(8, 14385)
(8, 8374)


University     Algoma                                             \
Country    Bangladesh                  Canada                      
Program      Business Computer science   Arts Biologies Business   
Year                                                               
2012-01-01          1                1     35        45      250   
2013-01-01          2                1     34        50      263   
2014-01-01          2                1     27        59      281   
2015-01-01          2                1     29        57      233   
2016-01-01          2                1     27        58      222   
2017-01-01          1                1     29        47      188   
2018-01-01          4                1     29        49      220   
2019-01-01          2                1      9        44      304   

University                                                                 \
Country                                                                     
Program    Computer science Economics General studies Geographies History   
Year                                                                        
2012-01-01              115         4              52          12      32   
2013-01-01              143         4              56          11      27   
2014-01-01              114         7              47           9      21   
2015-01-01               96        11              37          13      22   
2016-01-01               90         7              32          10      17   
2017-01-01               78         4              33          12      16   
2018-01-01               59         9              31          11      16   
2019-01-01              116         2              25           4       9   

University  ...     York                                               \
Country     ...   Zambia Zimbabwe                                       
Program     ... Business     Arts Biologies Business Computer science   
Year        ...                                                         
2012-01-01  ...        1        2         1        7                1   
2013-01-01  ...        1        2         1        6                1   
2014-01-01  ...        1        2         1        6                1   
2015-01-01  ...        1        2         1        5                1   
2016-01-01  ...        1        1         1        4                1   
2017-01-01  ...        1        1         1        4                1   
2018-01-01  ...        1        1         1        3                1   
2019-01-01  ...        1        1         1        3                1   

University                                                           \
Country                                                               
Program    Health Science Healthcare Studies Linguistics Philosophy   
Year                                                                  
2012-01-01              2                  2           2          4   
2013-01-01              2                  2           2          3   
2014-01-01              2                  2           2          3   
2015-01-01              2                  2           1          3   
2016-01-01              1                  1           1          2   
2017-01-01              1                  1           1          2   
2018-01-01              1                  1           1          2   
2019-01-01              1                  1           1          1   

University                 
Country                    
Program    Social Studies  
Year                       
2012-01-01              5  
2013-01-01              4  
2014-01-01              4  
2015-01-01              3  
2016-01-01              3  
2017-01-01              2  
2018-01-01              2  
2019-01-01              1  

[8 rows x 8374 columns]

In [ ]:
# Extract the university, country, and program columns used to create the final master set
topfinal= pd.DataFrame()
a= np.array(model_acp_pivot.columns)
for name, column in model_acp_pivot.iteritems(): 
    for i in name: 
        topfinal= topfinal.append({'University':name[0],'Country':name[1],'Program':name[2]},ignore_index=True)
topfinal.drop_duplicates(keep='first',inplace=True)
topfinal= topfinal[['University','Country','Program']].reset_index(drop=True)
topfinal

,University,Country,Program
0,Algoma,Bangladesh,Business
1,Algoma,Bangladesh,Computer science
2,Algoma,Canada,Arts
3,Algoma,Canada,Biologies
4,Algoma,Canada,Business
...,...,...,...
8369,York,Zimbabwe,Health Science
8370,York,Zimbabwe,Healthcare Studies
8371,York,Zimbabwe,Linguistics
8372,York,Zimbabwe,Philosophy


In [ ]:
# Merge the columns with the relevant headcount for each combination of university, country, program, and year, this is done to convert the 2 dimensional table into a 1 dimensional table for analysis purpose
model_acp = pd.merge(model_acp, topfinal, on=['University','Country','Program'], how="outer", indicator=True)
model_acp = model_acp.loc[model_acp["_merge"] == "both"].drop(["_merge"], axis=1)
model_acp = model_acp.reset_index(drop=True)
model_acp

,University,Country,Program,Year,Headcount
0,Algoma,Bangladesh,Business,2012-01-01,1
1,Algoma,Bangladesh,Business,2013-01-01,2
2,Algoma,Bangladesh,Business,2014-01-01,2
3,Algoma,Bangladesh,Business,2015-01-01,2
4,Algoma,Bangladesh,Business,2016-01-01,2
...,...,...,...,...,...
66987,York,Zimbabwe,Social Studies,2015-01-01,3
66988,York,Zimbabwe,Social Studies,2016-01-01,3
66989,York,Zimbabwe,Social Studies,2017-01-01,2
66990,York,Zimbabwe,Social Studies,2018-01-01,2


In [ ]:
#Extract the Ontario average headcount for each combination of country, program, and year (Used for visualization in tableau)
totals=model_acp.groupby(['Country','Program','Year'],as_index=False).agg(
    Average_Ontario=('Headcount','mean'))
totals['Average_Ontario'] = round(totals['Average_Ontario'],2)
totals

,Country,Program,Year,Average_Ontario
0,Afghanistan,Architectural Studies,2012-01-01,1.0
1,Afghanistan,Architectural Studies,2013-01-01,1.0
2,Afghanistan,Architectural Studies,2014-01-01,1.0
3,Afghanistan,Architectural Studies,2015-01-01,1.0
4,Afghanistan,Architectural Studies,2016-01-01,1.0
...,...,...,...,...
16395,Zimbabwe,Statistics,2015-01-01,1.0
16396,Zimbabwe,Statistics,2016-01-01,1.0
16397,Zimbabwe,Statistics,2017-01-01,1.0
16398,Zimbabwe,Statistics,2018-01-01,1.0


In [ ]:
#Join the Ontario average with the university headcount. Create a new column called "Type" used in Tableau to separate Graduate and Undergraduate studies
model_acp_undergrad = pd.merge(model_acp, totals, how="left", on=["Country","Program","Year"])
model_acp_undergrad["Type"] = "Undergraduate"
model_acp_undergrad

,University,Country,Program,Year,Headcount,Average_Ontario,Type
0,Algoma,Bangladesh,Business,2012-01-01,1,8.67,Undergraduate
1,Algoma,Bangladesh,Business,2013-01-01,2,8.87,Undergraduate
2,Algoma,Bangladesh,Business,2014-01-01,2,9.00,Undergraduate
3,Algoma,Bangladesh,Business,2015-01-01,2,9.07,Undergraduate
4,Algoma,Bangladesh,Business,2016-01-01,2,8.67,Undergraduate
...,...,...,...,...,...,...,...
66987,York,Zimbabwe,Social Studies,2015-01-01,3,1.50,Undergraduate
66988,York,Zimbabwe,Social Studies,2016-01-01,3,1.75,Undergraduate
66989,York,Zimbabwe,Social Studies,2017-01-01,2,1.50,Undergraduate
66990,York,Zimbabwe,Social Studies,2018-01-01,2,1.50,Undergraduate


##Concatenate and Extract Final data

In [ ]:
#Concatenate both Undergraduate and Graduate datasets
model_acp_final = pd.concat([model_acp_grad, model_acp_undergrad], ignore_index=True)
model_acp_final

,University,Country,Program,Year,Headcount,Average_Ontario,Type
0,Brock,Bangladesh,Business,2012-01-01,2,3.77,Graduate
1,Brock,Bangladesh,Business,2013-01-01,2,4.38,Graduate
2,Brock,Bangladesh,Business,2014-01-01,2,4.54,Graduate
3,Brock,Bangladesh,Business,2015-01-01,2,4.31,Graduate
4,Brock,Bangladesh,Business,2016-01-01,5,4.46,Graduate
...,...,...,...,...,...,...,...
98027,York,Zimbabwe,Social Studies,2015-01-01,3,1.50,Undergraduate
98028,York,Zimbabwe,Social Studies,2016-01-01,3,1.75,Undergraduate
98029,York,Zimbabwe,Social Studies,2017-01-01,2,1.50,Undergraduate
98030,York,Zimbabwe,Social Studies,2018-01-01,2,1.50,Undergraduate


In [ ]:
#Extract the final dataframe to a csv file
model_acp_final.to_csv("ACP_Data.csv", index=False)

In [ ]:
#Download the final dataset that will be used for both Tableau and for the Google Extraction code
files.download('ACP_Data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>